## **i. Mount google drive**
---

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## **ii. Import the libraries**
---

In [2]:
import cv2
import sys
import os
import scipy
import matplotlib
import numpy as np
import pandas as pd
import plotly.graph_objects as go

## **iii. Append /iss/issm/data/ in your python path**
---

In [3]:
sys.path.append('/content/gdrive/My Drive/Colab Notebooks/Intelligent_Sense_Making/')
print("All the system paths:")
sys.path

All the system paths:


['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/gdrive/My Drive/Colab Notebooks/Intelligent_Sense_Making/']

## **iv. Change working directory to /iss/issm/data/**
---

In [4]:
os.chdir('/content/gdrive/My Drive/Colab Notebooks/Intelligent_Sense_Making/')
print("Current working directory:")
os.getcwd()

Current working directory:


'/content/gdrive/My Drive/Colab Notebooks/Intelligent_Sense_Making'

## **v. Set Plotly Template Theme**
---

In [5]:
import plotly.io as pio

pio.templates["draft"] = go.layout.Template(
    layout_annotations=[
        dict(
            name="draft watermark",
            text="CONFIDENTIAL",
            textangle=-30,
            opacity=0.1,
            font=dict(color="black", size=100),
            xref="paper",
            yref="paper",
            x=0.6,
            y=0.6,
            showarrow=False,
        )
    ]
)
pio.templates.default = "plotly_dark+draft"

## **1. Part 1: Perform peak detection**
---
* Step 1: Read in the ecg signals
* Step 2: Perform peak detection
* Step 3: Display detected peaks

In [6]:
# Type your code below 
                                                                                # Step 1
from scipy.signal import find_peaks as findPeaks
from plotly.subplots import make_subplots


l2D     = pd.read_csv('ecg2D.csv',
                      header=None)

# Chose 2nd Column within the Dataset as the interested signal
ECGs    = l2D[1].values
(Pks,_) = findPeaks(ECGs,prominence=0.5,distance=100)  

# Plotly Lineplot showing signal and its peak                                                                 # Step 3
fig = go.Figure()
fig.add_trace(go.Scatter(x=l2D.index, y=ECGs,
                    mode='lines',
                    name='Signal Lines',
                    line=dict(color='lightskyblue', width=2)))
fig.add_trace(go.Scatter(x=Pks, y=ECGs[Pks],
                    mode='markers',
                    name='Peaks',
                    line=dict(color='lightsalmon', width=2)))
fig.update_layout(title='ECG Heartbeat Signal',
                   xaxis_title='Signal',
                   yaxis_title='Signal Magnitude')
fig.update_xaxes(title_font=dict(size=18, color='aqua'))
fig.update_yaxes(title_font=dict(size=18, color='aqua'))

fig.show()

## **Part 2: Baseline correction**
---
* Step 1: Create the baseline correction function
* Step 2: Perform baseline correction
* Step 3: Display the output

In [7]:
# Type your code below                                                      
                                                                                # Step 1
from scipy import sparse
from scipy.sparse.linalg import spsolve


def alsbase(y, lam, p, niter=10):
    L = len(y)
    D = sparse.diags([1,-2,1],[0,-1,-2], shape=(L,L-2))
    D = lam * D.dot(D.transpose()) # Precompute this term since it does not depend on `w`
    w = np.ones(L)
    W = sparse.spdiags(w, 0, L, L)
    for i in range(niter):
        W.setdiag(w) # Do not create a new matrix, just update diagonal values
        Z = W + D
        z = spsolve(Z, w*y)
        w = p * (y > z) + (1-p) * (y < z)
    return z  
                                                               # Step 2
ecgbase = alsbase(ECGs, 10^5,0.000005,niter=50)     
                                # 10^5 for lam, 0.000005 for p, 50 for iter
ecgcorr = ECGs-ecgbase
(Pks2,_) = findPeaks(ecgcorr,prominence=0.5,distance=100)  


fig = make_subplots(rows=2, cols=1, start_cell="top-left",
                    subplot_titles=("ECG Corrected Signal", "ECG Original Signal"),
                    shared_xaxes=True, shared_yaxes=True,
                    x_title='Signal', y_title='Signal Magnitude')

fig.add_trace(go.Scatter(x=l2D.index, y=ecgcorr,
                    mode='lines',
                    name='ECG Corrected',
                    line=dict(color='firebrick', width=1.5)),
              row=1, col=1)
fig.add_trace(go.Scatter(x=Pks2, y=ecgcorr[Pks2],
                    mode='markers',
                    name='Corrected Peaks',
                    line=dict(color='lightsalmon', width=2)),
              row=1, col=1)

fig.add_trace(go.Scatter(x=l2D.index, y=ECGs, mode='lines',name='ECG Original'),
              row=2, col=1)
fig.add_trace(go.Scatter(x=l2D.index, y=ecgbase,mode='lines',name='ECG Baseline',
                         line=dict(color='yellow', width=1)),
              row=2, col=1) 

fig.update_layout(title_text='Comparing Original & Corrected Signals',height=800)
fig.show()
